In [1]:
import os
import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns

from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from statsmodels.stats.outliers_influence import variance_inflation_factor

# CS:GO Round Winner Classification


URL: https://www.kaggle.com/christianlillelund/csgo-round-winner-classification

In [2]:
df_cs = pd.read_csv(os.path.join('dataset', 'csgo_round_snapshots.csv'))
df_cs.shape

(122410, 97)

In [3]:
df_cs.head()

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


In [4]:
df_cs.describe()

,time_left,ct_score,t_score,ct_health,t_health,ct_armor,t_armor,ct_money,t_money,ct_helmets,...,ct_grenade_flashbang,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade
count,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,...,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000,122410.000000
mean,97.886922,6.709239,6.780435,412.106568,402.714500,314.142121,298.444670,9789.023773,11241.036680,2.053901,...,1.853157,1.858100,1.540814,1.627146,1.001969,0.019819,0.048011,1.352095,0.027694,0.025750
std,54.465238,4.790362,4.823543,132.293290,139.919033,171.029736,174.576545,11215.042286,12162.806759,1.841470,...,1.772791,1.794473,1.737804,1.829147,1.458084,0.143933,0.227669,1.663246,0.169531,0.164162
min,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,54.920000,3.000000,3.000000,350.000000,322.000000,194.000000,174.000000,1300.000000,1550.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,94.910000,6.000000,6.000000,500.000000,500.000000,377.000000,334.000000,5500.000000,7150.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
75%,166.917500,10.000000,10.000000,500.000000,500.000000,486.000000,468.000000,14600.000000,18000.000000,4.000000,...,3.000000,3.000000,3.000000,3.000000,2.000000,0.000000,0.000000,2.000000,0.000000,0.000000
max,175.000000,32.000000,33.000000,500.000000,600.000000,500.000000,500.000000,80000.000000,80000.000000,5.000000,...,7.000000,7.000000,6.000000,9.000000,5.000000,3.000000,3.000000,5.000000,3.000000,2.000000


In [5]:
df_cs.replace('CT', 1, inplace=True)
df_cs.replace('T', 0, inplace=True)
df_cs.drop('map', axis=1, inplace=True)
df_cs['bomb_planted'] = df_cs['bomb_planted'].astype(int)

Y = df_cs['round_winner']
scaler = StandardScaler()
X = scaler.fit_transform(df_cs.drop('round_winner', axis=1))

df_cs_scaled = pd.DataFrame(X, columns=df_cs.drop('round_winner', axis=1).columns)
df_cs_scaled['round_winner'] = Y

In [111]:
vif = [variance_inflation_factor(df_cs_scaled.values, i) for i in range(len(df_cs_scaled.columns) - 1)]

/Users/tomasmikula/.virtualenvs/fcapy/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1717: RuntimeWarning: invalid value encountered in double_scalars
  return 1 - self.ssr/self.uncentered_tss


[5.365212886261656,
 1.98598384833828,
 1.9610539057347693,
 1.7940849048534948,
 23.628749822841456,
 17.910223046417947,
 5.94225881210855,
 8.701519714841144,
 2.6331244239682423,
 2.416300678248372,
 5.194022800946561,
 21.33225212641366,
 4.253790165549808,
 138.35145708792135,
 116.07894300264981,
 1.8542577660422255,
 9.178838822200774,
 1.5215461186661774,
 1.0432122023972443,
 2.2589875252764116,
 2.03372843621299,
 nan,
 1.0100282491362385,
 17.18432974373097,
 10.836979630941554,
 1.3319543821985982,
 1.1232670708037682,
 1.4190213629565314,
 1.0370937650928687,
 nan,
 1.011545398322502,
 1.0331216562185002,
 1.777145503988405,
 1.5906384949649193,
 178.75040106926372,
 1.000542774995635,
 nan,
 1.1903905118302678,
 1.0153727414247868,
 4.53812018028911,
 1.1945333057613665,
 1.030888560162447,
 1.9944496284215993,
 1.0206584869275028,
 1.002365591276979,
 1.0125207306683734,
 1.0382217176478967,
 1.0111609813518019,
 1.0203040594744224,
 1.7310426348677712,
 1.0459991848764

In [161]:
vif_df = pd.DataFrame(vif, columns=['vif'])
vif_df['features'] = df_cs_scaled.columns[:-1]
vif_remove = vif_df[vif_df['vif'] > 5].sort_values(['vif'], ascending=False)

df_cs_scaled = df_cs_scaled.drop(vif_remove['features'], axis=1)

vif_remove

,vif,features
75,204.103250,ct_weapon_usps
34,178.750401,t_weapon_glock
13,138.351457,ct_players_alive
14,116.078943,t_players_alive
71,57.694888,ct_weapon_deagle
72,46.174426,t_weapon_deagle
77,24.255713,ct_weapon_p250
78,23.880928,t_weapon_p250
4,23.628750,ct_health
11,21.332252,t_helmets


In [162]:
df_cs_scaled

,ct_score,t_score,bomb_planted,ct_weapon_ak47,ct_weapon_aug,t_weapon_aug,ct_weapon_bizon,t_weapon_bizon,ct_weapon_elite,t_weapon_elite,...,t_weapon_xm1014,t_weapon_fiveseven,t_weapon_p2000,ct_weapon_tec9,t_grenade_hegrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,-1.400576,-1.405702,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,1
1,-1.400576,-1.405702,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,1
2,-1.400576,-1.405702,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,1
3,-1.400576,-1.405702,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,13.301747,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,1
4,-1.191823,-1.405702,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122405,0.895711,1.496741,2.818773,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,0
122406,0.895711,1.704058,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,0
122407,0.895711,1.704058,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,0
122408,0.895711,1.704058,-0.354764,-0.470136,-0.292446,-0.083519,0.0,-0.009039,-0.063461,-0.042141,...,-0.01617,-0.093784,-0.065959,-0.085586,-0.542299,-0.137694,-0.210881,-0.163356,-0.156855,0


In [6]:
# default 0.25 test
train, test = train_test_split(df_cs_scaled)

# p > 1 scaled

In [167]:
train_input_var = train.drop(['round_winner'], axis=1)
train_input_target = train['round_winner'].astype(int)

test_input_var = test.drop(['round_winner'], axis=1)
test_input_target = test['round_winner'].astype(int)

In [172]:
model = QuadraticDiscriminantAnalysis()
model = model.fit(train_input_var, train_input_target)

/Users/tomasmikula/.virtualenvs/fcapy/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:715: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


## Train eval

In [173]:
model.score(train_input_var, train_input_target)

0.5183700589279685

In [174]:
confusion_matrix(train_input_target, model.predict(train_input_var).astype(float))

array([[39717,  7180],
       [37037,  7873]])

## Test eval

In [175]:
model.score(test_input_var, test_input_target)

0.511191713230729

In [176]:
confusion_matrix(test_input_target, model.predict(test_input_var).astype(float))

array([[13005,  2504],
       [12455,  2639]])